# DOM Events

## What are  DOM events?

In a nutshell, they are keyboard or mouse events in the browser. DOM is short for *Document Object Model*, the representation of the components of a web page displayed in a browser.  When you interact with something in a browser (like a notebook cell) events corresponding to keypresses and mouse clicks are generated by the browser and handled by the notebook's JavaScript. 

## The `DOMListener` widget

`DOMListener` is a widget that represents and provides a Python interface to mouse and keyboard interactions with the widgets, like `Image` or `Label`, that are displayed in the browser.

Unlike most other widgets, the `DOMListener` is not itself displayed. 

In [1]:
from ipywidgets import Label, HTML, HBox, Image, VBox, Box
from ipyevents import Event as DOMListener
from IPython.display import display

### Short example

The example below attaches a listener that responds to mouse clicks and key down on an `Label` widget and displays the event information in a `HTML` widget.

Note that the information returned in the event depends on whether the event was a mouse or a keyboard event.

In [2]:
l = Label('Click or type on me!')
l.layout.border = '2px solid red'

h = HTML('Event info')
d = DOMListener(source=l, watched_events=['click', 'keydown', 'mouseenter'])

def handle_event(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    h.value = content

d.on_dom_event(handle_event)
                            
display(l, h)

Label(value='Click or type on me!', layout=Layout(border='2px solid red'))

HTML(value='Event info')

### If you watch keyboard events then key presses are *not* passed on to the notebook 

This is to prevent the user from, e.g., inadvertantly cutting a cell by pressing `x` over a widget that is catching key events.

### Adding a new view of a widget attaches listeners to the view

Clicking on the label below will update the HTML widget above, just like clicking (or typing) on the first view of the widget did.

In [3]:
l

Label(value='Click or type on me!', layout=Layout(border='2px solid red'))

## Watchable events

You can get a list of the events to which a `DOMListener` can respond from the `supported_key_events` and `supported_mouse_events` properties.

In [4]:
print('Key events: ', d.supported_key_events)
print('Mouse events: ', d.supported_mouse_events)

Key events:  ['keydown', 'keyup']
Mouse events:  ['click', 'auxclick', 'dblclick', 'mouseenter', 'mouseleave', 'mousedown', 'mouseup', 'mousemove', 'wheel', 'contextmenu', 'dragstart', 'drag', 'dragend', 'dragenter', 'dragover', 'dragleave', 'drop']


### More about the watchable events

For more about these events, see the [MDN list of DOM events](https://developer.mozilla.org/en-US/docs/Web/Events)

**Keyboard events**

The keyboard events are triggered only if the mouse is over the widget.
+ `keydown` is triggered when a key is pressed down or if the key is held down and repeats.
+ `keyup` is triggered when a key is released.

**Mouse events**

Mouse events trigger only when the mouse is over the watched widget. 

+ `click` currently, triggered if any mouse button is pushed, but in the near future will trigger only when the primary mouse button is clicked.
+ `auxclick` triggers when a non-primary mouse button is clicked.
+ `dblclick` triggers when there is a double click on the element.
+ `mouseenter` triggers when the mouse enters the element.
+ `mouseleave` triggers when the mouse leaves the element.
+ `mousedown` triggers when any mouse button is depressed.
+ `mouseup` triggers when any mouse button is released.
+ `mousemove` triggers every time the mouse moves over the widget. *Enabling this will generate a large volume of events between the front end and  the back end.*
+ `wheel` trigger when the user scrolls in any direction. *Note that watching this disables scrolling of the notebook while the mouse is over the watched widget.*


### Optional filter useful if watching keyboard and mouse events

The `DOMListener` widget has a property, `ignore_modifier_key_events`, which, if set to `True`, causes key events for the modifier keys `Shift`, `Control`, `Alt`,  and `Meta`, to be ignored. This is useful if you want to watch keyboard events and mouse events, and you want to handle mouse events with modifier keys pressed differently than plain mouse events.

Normally, an action like Shift-click generates *two* events: a `keydown` for the `Shift` followed by a `click` (with its `shiftKey` set to `True`). While the unwanted `Shift` event could be filtered out on the python side, `ignore_modifier_key_events` provides a convenient way to avoid the extra events.

## Choose wisely

Think carefully about which events to monitor and experiment frequently. If, for example, you want to monitor both single and double mouse clicks, you probably do *not* want to watch for both `click` and `dblclick`. The reason is that when you double click a long string of events is generated:

```
mousedown
mouseup
click
mousedown
mouseup
click
dblclick
```

If you watch both `click` and `dblclick` then you will get three events per double click. 

One way to handle this case would be to watch only for `click`, and to check the `timeStamp` property of the event dictionary to see how close together in time the clicks are. If they are close enough, consider two consecutive clicks to be a double click, otherwise consider it to be two separate clicks. `timeStamp` is measured in milliseconds.

## The `event` dictionary

The single argument passed to a callback registered with `on_dom_event` is a dictionary. The entries in the dictionary depend on what kind of event occurs, but always contains the key `type` whose value is the type of event , e.g. `'click'` or `'keydown`'.

### Common to all events

All events have these keys in the event dictionary:

+ `altKey`: `bool`, `True` if the `alt` key was down when the event occurred, otherwise `False`.
+ `ctrlKey`: `bool`, `True` if the `control` key was down when the event occurred, otherwise `False`.
+ `metaKey`: `bool`, `True` if the `meta` key (the Windows key on Windows, the command key (cloverleaf) on the mac) was down when the event occurred, otherwise `False`.
+ `shiftKey`: `bool`, `True` if the `Shift` key was down when the event occurred, otherwise `False`.
+ `type`: `str`, type of event (e.g. `'click'`) that occurred.
+ `timeStamp`: `float`, the time (in milliseconds) at which the event occurred. 

### For keyboard events only

A key-related event has these keys in addition to the common ones above:

+ `key`: `str`, the key that was pressed. Note that if the shift key was held down when the key was pressed, and the key pressed was a letter, this will be an upper case letter.
+ `code`: `str`, also indicates the key pressed. the value begins with the string `Key`. This may contain additional information about the keypress, for example whether it was the left or right key pressed if the key appears multiple times on the keyboard.
+ `location`: `int`, the location of the key on the device (typically `0`).
+ `repeat`:  `bool`,  `True` if this event is the result of a key being held down.

For more details, see the [MDN documentation of HTML keyboard events](https://developer.mozilla.org/en-US/docs/Web/API/KeyboardEvent#Properties)

### For mouse events only

A mouse-related event has these keys in addition to the common ones above. They are broken into groups below to simplify the discussion. For more details about these properties, see the [MDN documentation of mouse events](https://developer.mozilla.org/en-US/docs/Web/API/MouseEvent).

**Which mouse button was pressed**
+ `button`: The button that was pushed or released for an event related to a mouse button push.
+ `buttons`: The buttons that were depressed when a mouse-related event occurs whether or not the event was a click. For example, when the mouse moves onto a widget being watched for `mouseenter`, then `buttons` indicates which, if any, mouse buttons are being help down.

**Location of the mouse event**

All locations are integers.

These attributes are standard properties of HTML mouse events.

+ `screenX`, `screenY`: Coordinates of the event relative to the user's whole screen.
+ `clientX`, `clientY` and `x`, `y`: Coordinates of the event relateive to the visible part of the web page in the browser.  
+ `pageX`, `pageY`: Coordinates of the event relative to the top of the page currently being displayed.
+ `offsetX`, `offsetY`: Offset of this DOM element relative to its parents.
+ `relativeX`, `relativeY`: The location of the mouse relative to the current element (i.e. widget). This emulates the functinoality of the `layerX/Y` attributes implemented in many browsers but not part of any standard.

**Additional location for some types of widgets**
+ `arrayX`, `arrayY`: Position of the mouse event in the underlying data object. For example, a click on an `Image` widget returns as `arrayX` and `arrayY` the coordinates of the click in the image. This location should be regarded as the *approximate* location the user intended given the difficulty of clicking accurately at the pixel level.

For widgets (like `Label`) for which there is no underlying array object these properties are not returned.

### Mouse scroll events only

If the event type is `wheel` then these keywords are also part of the event dictinoary:

+ `deltaX`: Horizontal scroll amount.
+ `deltaY`: Vertical scroll amount.
+ `deltaZ`: Scroll amount in z-direction.
+ `deltaMode`: Units of the scroll amount (0 is pixels, 1 is lines, 2 is pages).

## Preventing default actions for DOM events

Some DOM events, like a right-click on a mouse, have a default action that usually occurs; in the case of a right-click a context menu is typically displayed. If you wish to suppress the default action while watching a widget, set the `prevent_default_action` property to `True`.

Note that the DOM event for a right-click is `contextmenu`

In [5]:
label_no_default = Label('Right-click here and NO contextmenu is displayed')
label_no_default.layout.border = '2px solid red'

label_default = Label('Right-click here and contextmenu IS displayed')
label_default.layout.border = '2px solid blue'

h2 = HTML('Event info')

dom_no_default = DOMListener(source=label_no_default, 
                             watched_events=['contextmenu'], 
                             prevent_default_action=True)
dom_default = DOMListener(source=label_default, 
                          watched_events=['contextmenu'], 
                          prevent_default_action=False)

def handle_event_default_demo(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    h2.value = content

dom_no_default.on_dom_event(handle_event_default_demo)
dom_default.on_dom_event(handle_event_default_demo)
                            
display(label_no_default, h2, label_default)

Label(value='Right-click here and NO contextmenu is displayed', layout=Layout(border='2px solid red'))

HTML(value='Event info')

Label(value='Right-click here and contextmenu IS displayed', layout=Layout(border='2px solid blue'))

## Mouse location on an `Image` widget

The `Image` widget provides the location of mouse events in "array" or image coordinates as the `arrayX` and `arrayY` entries in the event dictionary.

Those entries take into account the size of the underlying image and the displayed width/height (including border and/or padding). 

Consider the image below, which has an intrinsic width of 600 pixels and height of 300 pixels. It is a gaussian centered in the image, i.e. with center coordinates 300, 150 in the original image.

![image of gaussian](images/gaussian_600_x_300.png)

First, let's load the image into an `Image` widget and attach a `DOMListener` to it. 

The `dragstart` event is added just to prevent the image from being dragged if a user clicks, heolds the mouse, and moves it.

In [6]:
with open('gaussian_600_x_300.png', 'rb') as f:
    value = f.read()

image = Image(value=value, format='png')

im_events = DOMListener()
im_events.source = image
im_events.watched_events = ['click']

no_drag = DOMListener(source=image, watched_events=['dragstart'], prevent_default_action = True)

The image is displayed in three different ways, none of which match the original image dimensions, and for all of which the aspect ratio is incorrect (the images are placed inside a `Box` to control their size independently).

Clicking on the center of *any* of the views of the image returns the same coordinate.

In [7]:
vbox = VBox()
vbox.layout.width = "100%"
header = HTML('<h2>Same image, three views</h2>')

images = HBox()
i1 = Box()
i1.layout.height = '100px'
i1.children = [image]

i2 = Box()
i2.children = [image]
i2.layout.width = '100px'

i3 = Box()
i3.layout.border = '5px red solid'
i3.layout.padding = '10px'
i3.layout.width = '300px'
i3.children = [image]

images.children = [i1, i2, i3]

coordinates = HTML('<h3>Click an image to see the click coordinates here</h3>')

vbox.children = [header, images, coordinates]

def update_coords(event):
    coordinates.value = '<h3>Clicked at ({}, {})</h3>'.format(event['arrayX'], event['arrayY'])
    
im_events.on_dom_event(update_coords)

vbox

VBox(children=(HTML(value='<h2>Same image, three views</h2>'), HBox(children=(Box(children=(Image(value=b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://assets-cdn.github.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-1c2c316b7a17f15536c6a26ed744654fc228a24658a7ae395cdcbf8329c8406b.css" media="all" rel="stylesheet" />\n  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-8100b9bf1eb6ed8b38eaad2fe7ba51d1895aa0602aafe4a87068d444e07e8c5c.css" media="all" rel="stylesheet" />\n  \n  \n  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/site-44b6bba3881278f33c221b6526379b55fbd098af3e553f54e81cab4c9a517c8e.css" media="all" rel="stylesheet" />\n  \n\n  <meta name="viewport" content="width=device-width">\n  \n  <title>ipyevents/gaussian_600_x_300.png at master \xc2\xb7 mwcraig/ipyevents \xc2\xb7 GitHub</title>\n  <link rel="search" type="application/opensearchdescription+xml" href="/opensearch.xml" title="GitHub">\n  <link rel="fluid-icon" href="https://github.com/fluidicon.png" title="GitHub">\n  <meta property="fb:app_id" content="1401488693436528">\n\n    \n    <meta content="https://avatars2.githubusercontent.com/u/1147167?s=400&amp;v=4" property="og:image" /><meta content="GitHub" property="og:site_name" /><meta content="object" property="og:type" /><meta content="mwcraig/ipyevents" property="og:title" /><meta content="https://github.com/mwcraig/ipyevents" property="og:url" /><meta content="ipyevents - A custom widget for returning mouse and keyboard events to Python" property="og:description" />\n\n  <link rel="assets" href="https://assets-cdn.github.com/">\n  \n  <meta name="pjax-timeout" content="1000">\n  \n  <meta name="request-id" content="9C2C:671B:448D9C:7C95C1:5A3A76F8" data-pjax-transient>\n  \n\n  <meta name="selected-link" value="repo_source" data-pjax-transient>\n\n    <meta name="google-site-verification" content="KT5gs8h0wvaagLKAVWq8bbeNwnZZK1r1XQysX3xurLU">\n  <meta name="google-site-verification" content="ZzhVyEFwb7w3e0-uOTltm8Jsck2F5StVihD0exw2fsA">\n  <meta name="google-site-verification" content="GXs5KoUUkNCoaAZn7wPN-t01Pywp9M3sEjnt_3_ZWPc">\n    <meta name="google-analytics" content="UA-3769691-2">\n\n<meta content="collector.githubapp.com" name="octolytics-host" /><meta content="github" name="octolytics-app-id" /><meta content="https://collector.githubapp.com/github-external/browser_event" name="octolytics-event-url" /><meta content="9C2C:671B:448D9C:7C95C1:5A3A76F8" name="octolytics-dimension-request_id" /><meta content="iad" name="octolytics-dimension-region_edge" /><meta content="iad" name="octolytics-dimension-region_render" />\n<meta content="/&lt;user-name&gt;/&lt;repo-name&gt;/blob/show" data-pjax-transient="true" name="analytics-location" />\n\n\n\n\n  <meta class="js-ga-set" name="dimension1" content="Logged Out">\n\n\n  \n\n      <meta name="hostname" content="github.com">\n  <meta name="user-login" content="">\n\n      <meta name="expected-hostname" content="github.com">\n    <meta name="js-proxy-site-detection-payload" content="MTY5MzBmY2FiMWViMDA5YzYyZTJhZGU2YjFkZWEzNmFiNjcwM2VkNzI2YjQwYjllNGUxZGQ1OGY2ZTE2NTEwYXx7InJlbW90ZV9hZGRyZXNzIjoiMTM0LjIxNC41LjI3IiwicmVxdWVzdF9pZCI6IjlDMkM6NjcxQjo0NDhEOUM6N0M5NUMxOjVBM0E3NkY4IiwidGltZXN0YW1wIjoxNTEzNzgwOTg1LCJob3N0IjoiZ2l0aHViLmNvbSJ9">\n\n\n  <meta name="html-safe-nonce" content="0f464325338a5cb7445fc5705a94a0fa30cf20d4">\n\n  <met